# Forensic Fraud Detection — Full Step-by-Step Notebook

This notebook is organized to follow the exact steps you requested. Run cells in order.

## 1) Load & Explore Data

Read the CSV and inspect basic info.

In [ ]:

import pandas as pd, numpy as np
df_raw = pd.read_csv("fraud_data.csv", dtype=str)
print("Shape:", df_raw.shape)
display(df_raw.head())
display(df_raw.dtypes)


## 2) Show first rows (parsed numeric columns)

In [ ]:

df = df_raw.copy()
for c in ['amt','lat','long','city_pop','merch_lat','merch_long']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
display(df.head())


## 3) Summary (normal vs fraud counts)
Clean the is_fraud column and show counts.

In [ ]:

import re
def extract_leading_digit(x):
    s = str(x)
    m = re.search(r'([01])', s)
    return int(m.group(1)) if m else np.nan
target = 'is_fraud' if 'is_fraud' in df.columns else df.columns[-1]
df['is_fraud_clean'] = df[target].apply(extract_leading_digit)
print("Counts:")
print(df['is_fraud_clean'].value_counts(dropna=False))
print("Percent:")
print(df['is_fraud_clean'].value_counts(normalize=True, dropna=False)*100)


## 4) Visualization of fraud vs normal (bar chart)

In [ ]:

import matplotlib.pyplot as plt
counts = df['is_fraud_clean'].value_counts().sort_index()
labels = ['non-fraud (0)','fraud (1)']
values = [counts.get(0,0), counts.get(1,0)]
plt.figure(figsize=(6,4))
plt.bar(labels, values)
plt.title("Class Distribution: Fraud vs Non-Fraud")
plt.ylabel("Counts")
for i,v in enumerate(values):
    plt.text(i, v + max(values)*0.01, str(int(v)), ha='center')
plt.show()


## 5) Data Preprocessing
- Drop duplicates
- Handle missing values
- Parse transaction time
- Create hour_of_day and distance_km (if lat/lon present)

In [ ]:

# Drop duplicates and handle missing target rows
df_clean = df.drop_duplicates().copy()
df_clean = df_clean.dropna(subset=['is_fraud_clean']).reset_index(drop=True)
df_clean['is_fraud_clean'] = df_clean['is_fraud_clean'].astype(int)

# Parse datetime and extract hour/day
if 'trans_date_trans_time' in df_clean.columns:
    df_clean['trans_date_trans_time'] = pd.to_datetime(df_clean['trans_date_trans_time'], errors='coerce')
    df_clean['hour_of_day'] = df_clean['trans_date_trans_time'].dt.hour
    df_clean['day_of_week'] = df_clean['trans_date_trans_time'].dt.dayofweek

# Convert numeric columns
for c in ['amt','lat','long','city_pop','merch_lat','merch_long']:
    if c in df_clean.columns:
        df_clean[c] = pd.to_numeric(df_clean[c], errors='coerce')

# Create distance feature if lat/long available
import math
def haversine_km(lat1, lon1, lat2, lon2):
    try:
        if any(pd.isna([lat1, lon1, lat2, lon2])):
            return np.nan
        rlat1, rlon1, rlat2, rlon2 = map(math.radians, [lat1, lon1, lat2, lon2])
        dlat = rlat2 - rlat1
        dlon = rlon2 - rlon1
        a = math.sin(dlat/2)**2 + math.cos(rlat1)*math.cos(rlat2)*math.sin(dlon/2)**2
        c = 2*math.asin(math.sqrt(a))
        return 6371*c
    except:
        return np.nan

if set(['lat','long','merch_lat','merch_long']).issubset(set(df_clean.columns)):
    df_clean['distance_km'] = df_clean.apply(lambda r: haversine_km(r['lat'], r['long'], r['merch_lat'], r['merch_long']), axis=1)

print("Clean shape:", df_clean.shape)
display(df_clean.head())


## 6) Handle class imbalance
Two strategies:
A) class-weighted models
B) upsample minority in training set

We will prepare train/test split and build both variants.

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

feature_cols = [c for c in ['amt','hour_of_day','city_pop','distance_km','lat','long','merch_lat','merch_long'] if c in df_clean.columns]
df_model = df_clean.dropna(subset=feature_cols + ['is_fraud_clean']).reset_index(drop=True)
X = df_model[feature_cols]
y = df_model['is_fraud_clean']

print("Features used:", feature_cols)
print("Dataset for modeling:", X.shape)
print("Class counts:", y.value_counts())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train/test shapes:", X_train.shape, X_test.shape)
print("Train class counts:", y_train.value_counts())

# Upsample minority in train
train_df = X_train.copy(); train_df['is_fraud_clean'] = y_train.values
minority = train_df[train_df['is_fraud_clean']==1]
majority = train_df[train_df['is_fraud_clean']==0]
if len(minority) > 0 and len(minority) < len(majority):
    minority_upsampled = resample(minority, replace=True, n_samples=len(majority), random_state=42)
    train_balanced = pd.concat([majority, minority_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)
    X_train_bal = train_balanced.drop(columns=['is_fraud_clean'])
    y_train_bal = train_balanced['is_fraud_clean']
    print("Balanced train created. New counts:", y_train_bal.value_counts())
else:
    X_train_bal, y_train_bal = X_train, y_train
    print("No upsampling performed (minority absent or not smaller).")


## 7) Feature scaling
StandardScaler fit on training set and applied to test set. For the balanced train set we scale it too.

In [ ]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)
if 'X_train_bal' in globals() and X_train_bal is not None:
    X_train_bal_s = scaler.transform(X_train_bal) if X_train_bal.shape[1]==X_train.shape[1] else scaler.fit_transform(X_train_bal)
else:
    X_train_bal_s = None
print("Scaling ready. Example means (should be near 0):", X_train_s.mean(axis=0)[:5])


## 8) Fraud Analysis — Transactions by amount and time

We compute statistics and show histograms for amount (fraud vs non-fraud) and transactions by hour of day.

In [ ]:

import matplotlib.pyplot as plt, numpy as np
if 'amt' in df_model.columns:
    stats = df_model.groupby('is_fraud_clean')['amt'].agg(['count','mean','median','std'])
    print("Amount stats by class:"); display(stats)
    nonfraud = df_model[df_model['is_fraud_clean']==0]['amt'].dropna()
    fraud = df_model[df_model['is_fraud_clean']==1]['amt'].dropna()
    # Non-fraud histogram
    plt.figure(figsize=(8,4))
    plt.hist(nonfraud, bins=60)
    plt.title("Amount distribution - Non-Fraud")
    plt.xlabel("Amount")
    plt.xlim(0, np.percentile(nonfraud,99))
    plt.show()
    # Fraud histogram
    plt.figure(figsize=(8,4))
    plt.hist(fraud, bins=60)
    plt.title("Amount distribution - Fraud")
    plt.xlabel("Amount")
    plt.xlim(0, np.percentile(nonfraud,99))
    plt.show()
else:
    print("No 'amt' column available.")

# Transactions by hour
if 'hour_of_day' in df_model.columns:
    counts = df_model.groupby(['hour_of_day','is_fraud_clean']).size().unstack(fill_value=0)
    print("Counts by hour (first rows):"); display(counts.head(24))
    hours = np.arange(0,24)
    nonf = counts.get(0, pd.Series(0,index=hours)).reindex(hours, fill_value=0)
    fr = counts.get(1, pd.Series(0,index=hours)).reindex(hours, fill_value=0)
    plt.figure(figsize=(10,4))
    plt.plot(hours, nonf, marker='o', label='Non-Fraud (0)')
    plt.plot(hours, fr, marker='o', label='Fraud (1)')
    plt.xlabel("Hour of day"); plt.ylabel("Transaction count")
    plt.title("Transactions by Hour — Fraud vs Non-Fraud")
    plt.legend(); plt.grid(True)
    plt.show()
else:
    print("No hour_of_day column available.")


## 9) Modeling — Logistic Regression & Random Forest

Train and evaluate Logistic Regression and Random Forest (class-weighted) and Random Forest on upsampled data (if created).

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
results = {}

def fit_eval(name, model, Xtr, Xte, ytr, yte):
    model.fit(Xtr, ytr)
    ypred = model.predict(Xte)
    try:
        yprob = model.predict_proba(Xte)[:,1]
    except:
        try: yprob = model.decision_function(Xte)
        except: yprob = None
    print("=== Model:", name, "===")
    print(classification_report(yte, ypred, digits=4))
    cm = confusion_matrix(yte, ypred)
    print("Confusion matrix:
", cm)
    aucv = roc_auc_score(yte, yprob) if yprob is not None else None
    print("ROC AUC:", aucv)
    results[name] = {'model': model, 'auc': aucv}
    return results[name]

# Logistic Regression with class weights
lr = LogisticRegression(class_weight='balanced', solver='liblinear', max_iter=1000, random_state=42)
fit_eval('LogReg_class_weight', lr, X_train_s, X_test_s, y_train, y_test)

# Random Forest with class weight on original train
rf = RandomForestClassifier(n_estimators=150, class_weight='balanced_subsample', random_state=42, n_jobs=-1)
fit_eval('RF_class_weight', rf, X_train_s, X_test_s, y_train, y_test)

# Random Forest on upsampled balanced train (if present)
if X_train_bal_s is not None:
    rf2 = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
    fit_eval('RF_upsampled_train', rf2, X_train_bal_s, X_test_s, y_train_bal, y_test)
else:
    print("No balanced train present; skipped RF_upsampled_train")


## 10) Compare performance (metrics table + ROC curves)

In [ ]:

import pandas as pd
rows = []
for name, info in results.items():
    rows.append({'model':name, 'roc_auc':info['auc']})
metrics_df = pd.DataFrame(rows).set_index('model')
display(metrics_df)

# ROC curves for models that have probabilities
plt.figure(figsize=(7,6))
for name, info in results.items():
    model = info['model']
    try:
        yprob = model.predict_proba(X_test_s)[:,1]
    except:
        try: yprob = model.decision_function(X_test_s)
        except: yprob = None
    if yprob is not None:
        fpr, tpr, _ = roc_curve(y_test, yprob)
        plt.plot(fpr, tpr, label=f"{name} (AUC={info['auc']:.3f})")
plt.plot([0,1],[0,1], '--', linewidth=1)
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC Comparison")
plt.legend(); plt.grid(True); plt.show()


## 11) Visualization & Dashboard
All important graphs are in this notebook (above). We also save key charts to `outputs/` for use in README or a Power BI import.

In [ ]:

import os
outdir = "outputs"
os.makedirs(outdir, exist_ok=True)

# Save class distribution
plt.figure(figsize=(6,4))
counts = df_clean['is_fraud_clean'].value_counts().sort_index()
labels = ['non-fraud (0)','fraud (1)']
values = [counts.get(0,0), counts.get(1,0)]
plt.bar(labels, values); plt.title("Class Distribution: Fraud vs Non-Fraud")
plt.tight_layout(); plt.savefig(os.path.join(outdir, "class_distribution.png")); plt.close()

# Save example amount histograms if available
if 'amt' in df_model.columns:
    nonfraud = df_model[df_model['is_fraud_clean']==0]['amt'].dropna()
    fraud = df_model[df_model['is_fraud_clean']==1]['amt'].dropna()
    plt.figure(figsize=(8,4)); plt.hist(nonfraud, bins=60); plt.xlim(0, np.percentile(nonfraud,99))
    plt.tight_layout(); plt.savefig(os.path.join(outdir, "amount_nonfraud.png")); plt.close()
    plt.figure(figsize=(8,4)); plt.hist(fraud, bins=60); plt.xlim(0, np.percentile(nonfraud,99))
    plt.tight_layout(); plt.savefig(os.path.join(outdir, "amount_fraud.png")); plt.close()

# Save ROC plot
plt.figure(figsize=(7,6))
for name, info in results.items():
    model = info['model']
    try: yprob = model.predict_proba(X_test_s)[:,1]
    except:
        try: yprob = model.decision_function(X_test_s)
        except: yprob = None
    if yprob is not None:
        fpr,tpr,_ = roc_curve(y_test, yprob)
        plt.plot(fpr,tpr, label=f"{name} (AUC={info['auc']:.3f})")
plt.plot([0,1],[0,1],'--'); plt.legend(); plt.tight_layout(); plt.savefig(os.path.join(outdir, "roc_comparison.png")); plt.close()

print("Saved outputs to", outdir)


## 12) Documentation & next steps
- Add the images in `outputs/` into README.
- Optionally import the `outputs/*.png` into a Power BI report and create interactive visuals.

If you want, say **make CV** and provide your details (Name, email, phone, university, graduation year, certifications, languages) and I'll produce a professional CV that highlights this project.